## Quiz 05 - Parallel Computing, Reproducibility, and Containers

### Instructions

This quiz is based on the material covered in lectures 26 to 32. You may use
any resources available to you, including the lecture notes and the internet.

All the data required for this quiz can be found in the `data` folder within this repository. If you need to recreate the datasets, you can do so by running the Python script included in the `script-data-generation` folder.

**Important:** Please start by completing Question 01 to set up the correct Python environment before proceeding with the other questions.

This notebook contains the questions you need to answer.
If possible, please submit your answers as an `.html` file on Canvas.

### **Question 01: Setting up the Python Environment**

Before proceeding with the rest of the quiz, it is important to set up a Python environment with specific package versions to ensure compatibility and reproducibility. This quiz requires **Python 3.10** and the following packages with exact versions:
- `dask-sql=2024.5.0`
- `dask=2024.4.1`
- `ipykernel=6.29.3`
- `joblib=1.3.2`
- `numpy=1.26.4`
- `pandas=2.2.1`

You can use tools like `conda`, `pipenv`, or `uv` to manage your environment. If you use conda, please make sure you **create the environment and install all packages in the same command**. Also, make sure to change your current environment to the new environment after creation. 

Write the terminal commands to accomplish this in the code cell below:

In [1]:
# Please write your bash commands here. You can run them using the `!` operator or the `%%bash` magic.

conda create -n venv-dask -c conda-forge python=3.10 dask-sql=2024.5.0 dask=2024.4.1 ipykernel=6.29.3 joblib=1.3.2 numpy=1.26.4 pandas=2.2.1 tpot=0.12.2 prophet -y

SyntaxError: invalid syntax (231341539.py, line 3)

### Question 02 - Parallelising a Function with Joblib

Use `joblib` to parallelise the computation of squaring numbers in a large array. Import the required packages and write code that uses four cores to parallelise the computation. Print the first 10 numbers.

```python
import numpy as np

def square(x):
    return x ** 2

numbers = np.arange(1000000)
```

In [8]:
# Please write your answer here.
import numpy as np
from joblib import Parallel, delayed

def square(x):
    return x ** 2

numbers = np.arange(1000000)

result = Parallel(n_jobs=4)(delayed(square)(x) for x in numbers)
result[0:10]


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

### Question 03 - Using Dask Arrays for Large Data

Using Dask's `array` module, create a Dask array of random numbers with 10,000 rows and 10,000 columns. The array should be divided into chunks of 1,000 rows by 1,000 columns to enable efficient parallel computation. Populate the array with random numbers drawn from a normal distribution, where the mean is 0 and the standard deviation is 1. After creating the array, compute the mean, standard deviation, maximum, and minimum of the array using Dask's parallel computation capabilities. Use the `.compute()` method to execute the computations and print the results.

In [12]:
# Please write your answer here.
import numpy as np
import dask.dataframe as dd
import dask.array as da

numbers = np.random.standard_normal(size = (10000,10000))
da_a = da.from_array(numbers, chunks=(1000,1000))

print(da_a.mean().compute())
print(da_a.std().compute())
print(da_a.max().compute())
print(da_a.min().compute())

0.00015911072165366506
1.000041777284891
5.894120814154801
-5.878393050347428


### Question 04 - Dask DataFrame Operations with Parquet Files

The `data` folder contains datasets for four countries—Brazil, India, UK, and USA—covering the years 1945 to 2023. Each country's data is stored in a separate Parquet file named after the country (`Brazil.parquet`, `India.parquet`, `UK.parquet`, `USA.parquet`). Each file contains the following columns:

- `country` (string): The name of the country.
- `year` (integer): The year of the record.
- `gdp_per_capita` (float): The GDP per capita for that country and year.
- `population` (integer): The population for that country and year.

Using Dask's `dataframe` module, read _only the `country` and the `gdp_per_capita` columns_ from the Parquet files into a Dask DataFrame. Then, compute the mean and standard deviation of the GDP per capita for each country using Dask's parallel computation capabilities.

In [25]:
# Please write your answer here.
import dask

Brazil = dask.dataframe.read_parquet('data/Brazil.parquet', columns = ("country", "gdp_per_capita"))
India = dask.dataframe.read_parquet('data/India.parquet', columns = ("country", "gdp_per_capita"))
UK = dask.dataframe.read_parquet('data/UK.parquet', columns = ("country", "gdp_per_capita"))
USA = dask.dataframe.read_parquet('data/USA.parquet', columns = ("country", "gdp_per_capita"))

print("Brazil: " + str(Brazil["gdp_per_capita"].mean().compute()) + " and " + str(Brazil["gdp_per_capita"].std().compute()))
print("India: " + str(India["gdp_per_capita"].mean().compute()) + " and " + str(India["gdp_per_capita"].std().compute()))
print("UK: " + str(UK["gdp_per_capita"].mean().compute()) + " and " + str(UK["gdp_per_capita"].std().compute()))
print("USA: " + str(USA["gdp_per_capita"].mean().compute()) + " and " + str(USA["gdp_per_capita"].std().compute()))

Brazil: 5496.292030611491 and 2682.494157650465
India: 1251.704442866794 and 456.5256281158409
UK: 27496.851363019257 and 10607.858035765952
USA: 40189.822289795375 and 14892.455747057224


### Question 05 - Dask and SQL Queries

Load the `data.csv` file into a Dask DataFrame and use the `dask_sql` package to perform a SQL query that selects the `country` and `gdp_per_capita` columns and filters the rows where `gdp_per_capita` is greater than 20000 in 2014. Display the results. Do not forget to register the Dask DataFrame as a SQL table with the `create_table` method.

In [62]:
# Please write your answer here.

import dask.dataframe as dd
from dask_sql import Context

df = dd.read_csv("data/data.csv")
df = df.astype({"country": "string", "gdp_per_capita": "float64", "year": "int64"})
c = Context()
c.create_table("data", df)


query = """
SELECT country, gdp_per_capita
FROM data
WHERE gdp_per_capita > 20000 AND year = 2014
"""
c.sql(query).compute()



,country,gdp_per_capita
227,UK,40455.486012
306,USA,65386.141694


### Question 06 - Parallelising a Function with Dask Delayed

Suppose we need to compute the sum of squares of numbers for large ranges. The function below calculates the sum of squares from `0` up to `n-1`. Modify the given `sum_of_squares` function to use Dask's `@delayed` decorator and compute the sum of squares for each number in the numbers list in parallel. Measure and print the total execution time for the parallel computation, and print the results for each input number (as indicated in the code).

```python
import time

def sum_of_squares(n):
    """Compute the sum of squares from 0 to n-1."""
    return sum(i * i for i in range(n))

numbers = [100_000_000, 200_000_000, 300_000_000, 400_000_000]

# Measure the start time
start_time = time.time()

# Perform the computations serially
results_serial = []
for n in numbers:
    result = sum_of_squares(n)
    results_serial.append(result)
    print(f"Sum of squares up to {n}: {result}")

# Measure the end time
end_time = time.time()

# Calculate and print the total execution time
serial_execution_time = end_time - start_time
print(f"Total execution time (serial): {serial_execution_time:.2f} seconds")
```

In [77]:
# Please write your answer here.
import dask
import time

@dask.delayed
def sum_of_squares(n):
    """Compute the sum of squares from 0 to n-1."""
    return sum(i * i for i in range(n))

numbers = [100_000_000, 200_000_000, 300_000_000, 400_000_000]

# Measure the start time
start_time = time.time()

# Perform the computations serially
results_serial = []
for n in numbers:
    result = sum_of_squares(n)
    results_serial.append(result)
    print(f"Sum of squares up to {n}: {result}")

# Measure the end time
end_time = time.time()

# Calculate and print the total execution time
serial_execution_time = end_time - start_time
print(f"Total execution time (serial): {serial_execution_time:.2f} seconds")


start_time2 = time.time()
results = []
for n in numbers:
    results.append(sum_of_squares(n))
final_results = dask.compute(*results)
end_time2 = time.time()

for i in range(len(numbers)):
    print(f"Sum of squares up to {numbers[i]}: {final_results[i]}")

parallel_execution_time = end_time2 - start_time2
print(f"Total execution time (parallel): {parallel_execution_time:.2f} seconds")

Sum of squares up to 100000000: Delayed('sum_of_squares-2fc3948a-95be-41d8-94be-74f9f56a2c6b')
Sum of squares up to 200000000: Delayed('sum_of_squares-f05a89ea-48d7-49ba-8b24-fd4575be0d0c')
Sum of squares up to 300000000: Delayed('sum_of_squares-b1d720a6-fa91-4c9d-a94c-313367c1802d')
Sum of squares up to 400000000: Delayed('sum_of_squares-35332d5d-73a9-49c0-967a-d15c9a72ce5c')
Total execution time (serial): 0.00 seconds
Sum of squares up to 100000000: 333333328333333350000000
Sum of squares up to 200000000: 2666666646666666700000000
Sum of squares up to 300000000: 8999999955000000050000000
Sum of squares up to 400000000: 21333333253333333400000000
Total execution time (parallel): 60.52 seconds


### Question 07 - Using `pip` and `requirements.txt` for Dependency Management

Explain how you can use `pip` to manage dependencies in a Python project. Describe the process of generating a `requirements.txt` file from your current environment and how to use this file to install the same packages in another environment or on a different machine. Please comment your code to explain each step. It is not necessary to run the code, but you can if you want to test it. You can use the code cell below to write your answer.

In [ ]:
# Please write your answer here.

# You can use the command `pip freeze > requirements.txt` to generate a requirements.txt file that has a list of your installed packages with their versions, and then send the 
# requirements.txt file to another computer and install the dependencies on there using `pip install -r requirements.txt`. 

### Question 08 - Writing a Simple Dockerfile

Write a simple `Dockerfile` that creates a Docker image for a Python application. The application consists of a single Python script named `app.py` that prints "Hello, World!" when executed. The `Dockerfile` should use the official Python image as the base image, set a working directory in the container called `app`, and copy the `app.py` script into the image. When the container is run, it should execute the `app.py` script and print "Hello, World!".

#### Please write your answer here. You can use ```dockerfile to format your code.

```dockerfile
FROM python:3.11
WORKDIR /app
RUN echo "print('Hello, World!')" > app.py
COPY app.py
CMD ["python", "app.py"]
```

### Question 09 - Writing a Dockerfile to Install Software on a Base Image

Create a Dockerfile that starts from an Ubuntu 24.04 base image and installs the following software:

- Git version 2.43.0-1ubuntu7.1
- SQLite version 3.45.1-1ubuntu2

Ensure that you specify the exact versions of the packages by checking their versions after installation. Include commands to clean up the package manager cache after installation to reduce the image size.

#### Please write your answer here. You can use ```dockerfile to format your code.

```dockerfile
FROM ubuntu:24.04

RUN apt-get update && apt-get install -y --no-install-recommends\
    git=1:2.43.0-1ubuntu7.1 \
    sqlite3=3.45.1-1ubuntu2 \
    apt-get clean && rm -rf /var/lib/apt/lists/*
```

### Question 10 - Writing a Dockerfile to Install Python and Packages on Ubuntu

Write a `Dockerfile` that starts from an Ubuntu 24.04 base image, installs Python 3.12 and `pip`, and then uses `pip` to install specific versions of `numpy` (1.26.4), `pandas` (2.2.2), and `matplotlib` (3.9.2). Ensure you include commands to clean up the package manager cache after installation to reduce the image size. Set up a working directory named `app/` and configure the container to start an interactive Python shell `python3` by default.

#### Please write your answer here. You can use ```dockerfile to format your code.

```dockerfile
FROM ubuntu:24.04

WORKDIR /app

RUN apt-get update && apt-get install -y --no-install-recommends \
    python3.12=3.12.3-1ubuntu0.7 \
    python3-pip=24.0+dfsg-1ubuntu1.1 && \
    pip install numpy==1.26.4 pandas==2.2.2 matplotlib==3.9.2 && \
    apt-get clean && rm -rf /var/lib/apt/lists/*

CMD ["python3"]
```